<a href="https://colab.research.google.com/github/Abhijit-coder234/Sound-Classification-ML/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os
import zipfile
from sklearn.model_selection import train_test_split 
from os import path, getcwd, chdir
 
local_zip = '/Spectrograms.zip' 
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/Spectrograms')
zip_ref.close()
 
train_BareFoot_dir = os.path.join('/Spectrograms/Train/BareFoot')
train_Grass_dir = os.path.join('/Spectrograms/Train/Grass')
train_MetalSurface_dir = os.path.join('/Spectrograms/Train/MetalSurface')
 
Validations_BareFoot_dir = os.path.join('/Spectrograms/Validations/BareFoot')
Validations_Grass_dir = os.path.join('/Spectrograms/Validations/Grass')
Validations_MetalSurface_dir = os.path.join('/Spectrograms/Validations/MetalSurface')
 
train_BareFoot_names = os.listdir(train_BareFoot_dir)
print(train_BareFoot_names[:10])
train_Grass_names = os.listdir(train_Grass_dir)
print(train_Grass_names[:10])
train_MetalSurface_names = os.listdir(train_MetalSurface_dir)
print(train_MetalSurface_names[:10])
 
Validations_BareFoot_names = os.listdir(Validations_BareFoot_dir)
print(Validations_BareFoot_names[:10])
Validations_Grass_names = os.listdir(Validations_Grass_dir)
print(Validations_Grass_names[:10])
Validations_MetalSurface_names = os.listdir(Validations_MetalSurface_dir)
print(Validations_MetalSurface_names[:10])
 
print('total training BareFoot images:', len(os.listdir(train_BareFoot_dir)))
print('total training Grass images:', len(os.listdir(train_Grass_dir)))
print('total training MetalSurface images:', len(os.listdir(train_MetalSurface_dir)))
 
print('total validation BareFoot images:', len(os.listdir(Validations_BareFoot_dir)))
print('total validation Grass images:', len(os.listdir(Validations_Grass_dir)))
print('total validation MetalSurface images:', len(os.listdir(Validations_MetalSurface_dir)))

In [ ]:
    # DESIRED_ACCURACY = 0.90
 
    #  class myCallback(tf.keras.callbacks.Callback):
    #      def on_epoch_end(self, epoch, logs={}):
    #          if(logs.get('acc')>DESIRED_ACCURACY):
    #             print("Reached above 90.0% accuracy so cancelling training!")
    #             self.model.stop_training = True
        
 
    # callbacks = myCallback()
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(41, 60, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        #tf.keras.layers.MaxPooling2D(2,2),
        #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        #tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
 
    model.summary()
 
    from tensorflow.keras.optimizers import RMSprop
 
    model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])
 
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
    train_datagen = ImageDataGenerator(rescale=1/255)
    Validation_datagen = ImageDataGenerator(rescale=1/255)
 
    train_generator = train_datagen.flow_from_directory(
        '/Spectrograms/Train/',
        target_size=(40, 61),
        batch_size=10,
        class_mode='categorical')
    
    Validation_generator = Validation_datagen.flow_from_directory(
        '/Spectrograms/Validations/',
        target_size=(40,61),
        batch_size=10,
        class_mode='categorical')
  
    history = model.fit(train_generator, 
                                  steps_per_epoch=70, 
                                  epochs=15, 
                                  verbose=1, 
                                  validation_data = Validation_generator,
                                  validation_steps=24
                                  )

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 39, 58, 16)        448       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 19, 29, 16)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 17, 27, 16)        2320      
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 8, 13, 16)         0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 6, 11, 32)         4640      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 3, 5, 32)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 480)              

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(40, 61))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  if classes[0][0]==1.0:
    print("The audio is BareFoot")
  elif classes[0][1]==1.0:
    print("The audio is Grass")
  elif classes[0][2]==1.0:
    print("The audio is MetalSurface")
  else:
    print("The audio is not recognised")